# STEP 1

## 1. 정규화
## 2. 댓글에서 주요내용(동행자, 대기시간 등) 추출
## 3. 주요내용 빈도수
## 4. 형태소 추출
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/Crawling/지우/data

/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/Crawling/지우/data


# 정규 표현식 적용

In [ ]:
import pandas as pd
import numpy as np

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
path = '/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/Crawling/지우/data/label_data.csv'
pd.read_csv(path).head()

,Unnamed: 0,label,name,address,NAVER_name,category,review_num,reviews,review
0,0,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,하얗게 불태웠다
1,1,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,굿
2,2,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,이색적인곳
3,3,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,닭발 주먹밥 매콤하니 좋아요
4,4,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,월드컵 관람하러 들렀습니다. 넘 재밌었어요!!


In [ ]:
raw = pd.read_csv(path)

In [ ]:
df = raw.copy()

In [ ]:
df.columns

Index(['Unnamed: 0', 'label', 'name', 'address', 'NAVER_name', 'category',
       'review_num', 'reviews', 'review'],
      dtype='object')

In [ ]:
df = df[['label', 'name', 'address', 'NAVER_name', 'category',
       'review_num', 'reviews', 'review']]

df[[
    '예약 없이 이용', '예약 후 이용', '포장배달', '바로 입장', '10분 이내', '30분 이내',
    '30분 이상', '1시간 이상', '2시간 이상', '데이트', '친목', '나들이',
    '여행', '일상', '기념일', '회식', '가족모임', '비지니스', '연인・배우자',
    '친구', '지인・동료', '혼자', '아이', '부모님', '반려동물', '친척・형제', '기타'
]] = 0

In [ ]:
df.columns

Index(['label', 'name', 'address', 'NAVER_name', 'category', 'review_num',
       'reviews', 'review', '예약 없이 이용', '예약 후 이용', '포장배달', '바로 입장', '10분 이내',
       '30분 이내', '30분 이상', '1시간 이상', '2시간 이상', '데이트', '친목', '나들이', '여행', '일상',
       '기념일', '회식', '가족모임', '비지니스', '연인・배우자', '친구', '지인・동료', '혼자', '아이', '부모님',
       '반려동물', '친척・형제', '기타'],
      dtype='object')

In [ ]:
# review 컬럼에서 각 컬럼명 카운팅 및 대체 (앞 35글자만 적용)
def process_review(row):
    review_text = row['review'][:35]  # 앞 35글자만 가져오기

    for column in df.columns[8:]:  # 컬럼명 부분만 확인
        if column in review_text:
            review_text = review_text.replace(column, "")
            row[column] += 1  # 카운트 증가

    # 전체 리뷰에서 대체된 부분 반영
    row['review'] = row['review'].replace(row['review'][:35], review_text, 1).strip()  # 빈 문자열 정리
    return row

# 각 행에 대해 함수 적용
df = df.apply(process_review, axis=1)

df

,label,name,address,NAVER_name,category,review_num,reviews,review,예약 없이 이용,예약 후 이용,...,비지니스,연인・배우자,친구,지인・동료,혼자,아이,부모님,반려동물,친척・형제,기타
0,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,하얗게 불태웠다,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,굿,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,이색적인곳,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,닭발 주먹밥 매콤하니 좋아요,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,월드컵 관람하러 들렀습니다. 넘 재밌었어요!!,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160967,1,주식회사 알고기인제주,제주 제주시 도두일동 2623-4번지 1층,R고기 in Jeju,돼지고기구이,157,예약 후 이용 바로 입장 가족모임 아이 R고기는 사랑입니다❤️❤️❤️❤️❤️\n더운...,아는 사람만 가는 맛집이래요.\n외진 곳인데도 어찌 알고오셨는지\n손님들로 가득찼네...,0,0,...,0,0,0,0,0,0,0,0,0,0
160968,1,주식회사 알고기인제주,제주 제주시 도두일동 2623-4번지 1층,R고기 in Jeju,돼지고기구이,157,예약 후 이용 바로 입장 가족모임 아이 R고기는 사랑입니다❤️❤️❤️❤️❤️\n더운...,매월 하는 골프행사 마치고 숨은 맛집인 R고기 in 제주에서 뒷풀이 했습니다.\n많...,0,0,...,0,0,0,0,0,0,0,0,0,0
160969,1,주식회사 알고기인제주,제주 제주시 도두일동 2623-4번지 1층,R고기 in Jeju,돼지고기구이,157,예약 후 이용 바로 입장 가족모임 아이 R고기는 사랑입니다❤️❤️❤️❤️❤️\n더운...,재방문이에요 너무맛있어요\n삼겹 항정살 오무라이스 토마토스튜 맛없는게없어요 최고,0,0,...,0,0,0,0,0,0,0,0,0,0
160970,1,주식회사 알고기인제주,제주 제주시 도두일동 2623-4번지 1층,R고기 in Jeju,돼지고기구이,157,예약 후 이용 바로 입장 가족모임 아이 R고기는 사랑입니다❤️❤️❤️❤️❤️\n더운...,근처 숙소에 묵게 되어 가까운 고깃집을 찾다가 오게 되었어요! 혼자 갔지만 매우매우...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df[300:350]

,label,name,address,NAVER_name,category,review_num,reviews,review,예약 없이 이용,예약 후 이용,...,비지니스,연인・배우자,친구,지인・동료,혼자,아이,부모님,반려동물,친척・형제,기타
300,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,일본느낌 낭낭한 술집이 인제에 생겨서 부리나케 달려갔습니다🕺🏽\n8~9시즈음 방문하...,0,0,...,0,0,0,0,0,0,0,0,0,0
301,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,너무 친절하시고 즐겁게 마시다 갑니다,1,0,...,0,1,0,0,0,0,0,0,0,0
302,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,오꼬노미야키 생맥주 냠냠,1,0,...,0,1,0,0,0,0,0,0,0,0
303,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,닭날개가 아니고 봉이에요~\n엄청 쪼꼬맘 ㅎ\n사람 시끌벅적 2차로 드시기에 좋아보...,1,0,...,0,0,0,0,0,0,0,0,0,1
304,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,테바나카는 오리지널이 더 맛있는것 같아요.\n닭날개가 작아서 양은 그다지 많은 느낌...,1,0,...,0,0,1,0,0,0,0,0,0,0
305,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,", , 동네에 새로 생겨서 와봤는데 가성비 갑,,\n단체로 와서 엄청 시켜 먹었...",1,0,...,0,1,1,0,0,0,0,0,0,0
306,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,생마차 생겨서 좋아요🫶🏻\n짭쪼름하니 짱 맛있어요!\n2차로 한잔 더 하고 가요!!!,1,0,...,0,1,0,0,0,0,0,0,0,0
307,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,새로운 맛집이 생겼다고해서 방문했는데 닭날개튀김이랑 다른 안주류 음식들이 맛도 좋고...,1,0,...,0,1,0,0,0,0,0,0,0,0
308,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,", 안주도 맛있고 생맥도 맛있어요!!!\n다만 확실히 넓은 공간에 사람이 많으니 ...",1,0,...,0,0,1,1,0,0,0,0,0,0
309,1,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,가격이 저렴해요. 메뉴가 다양해서 좋았어요 술이 다양하게 있어서 좋았어요 👍,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 정규 표현식 함수 정의

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [ ]:
# 정규표현으로 특수 문자 제거 완료
# df의 이름은 따로 지정하신 데이터프레임 이름 사용
apply_regular_expression(df['review'][0])

'하얗게 불태웠다'

In [ ]:
df['review'] = df['review'].apply(lambda x: apply_regular_expression(x))

In [ ]:
df.columns

Index(['label', 'name', 'address', 'NAVER_name', 'category', 'review_num',
       'reviews', 'review', '예약 없이 이용', '예약 후 이용', '포장배달', '바로 입장', '10분 이내',
       '30분 이내', '30분 이상', '1시간 이상', '2시간 이상', '데이트', '친목', '나들이', '여행', '일상',
       '기념일', '회식', '가족모임', '비지니스', '연인・배우자', '친구', '지인・동료', '혼자', '아이', '부모님',
       '반려동물', '친척・형제', '기타'],
      dtype='object')

In [ ]:
df

,label,name,address,NAVER_name,category,review_num,reviews,review,예약 없이 이용,예약 후 이용,...,비지니스,연인・배우자,친구,지인・동료,혼자,아이,부모님,반려동물,친척・형제,기타
0,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,하얗게 불태웠다,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,굿,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,이색적인곳,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,닭발 주먹밥 매콤하니 좋아요,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,월드컵 관람하러 들렀습니다 넘 재밌었어요,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160967,1,주식회사 알고기인제주,제주 제주시 도두일동 2623-4번지 1층,R고기 in Jeju,돼지고기구이,157,예약 후 이용 바로 입장 가족모임 아이 R고기는 사랑입니다❤️❤️❤️❤️❤️\n더운...,아는 사람만 가는 맛집이래요외진 곳인데도 어찌 알고오셨는지손님들로 가득찼네요애들 데...,0,0,...,0,0,0,0,0,0,0,0,0,0
160968,1,주식회사 알고기인제주,제주 제주시 도두일동 2623-4번지 1층,R고기 in Jeju,돼지고기구이,157,예약 후 이용 바로 입장 가족모임 아이 R고기는 사랑입니다❤️❤️❤️❤️❤️\n더운...,매월 하는 골프행사 마치고 숨은 맛집인 고기 제주에서 뒷풀이 했습니다많은 인원을 ...,0,0,...,0,0,0,0,0,0,0,0,0,0
160969,1,주식회사 알고기인제주,제주 제주시 도두일동 2623-4번지 1층,R고기 in Jeju,돼지고기구이,157,예약 후 이용 바로 입장 가족모임 아이 R고기는 사랑입니다❤️❤️❤️❤️❤️\n더운...,재방문이에요 너무맛있어요삼겹 항정살 오무라이스 토마토스튜 맛없는게없어요 최고,0,0,...,0,0,0,0,0,0,0,0,0,0
160970,1,주식회사 알고기인제주,제주 제주시 도두일동 2623-4번지 1층,R고기 in Jeju,돼지고기구이,157,예약 후 이용 바로 입장 가족모임 아이 R고기는 사랑입니다❤️❤️❤️❤️❤️\n더운...,근처 숙소에 묵게 되어 가까운 고깃집을 찾다가 오게 되었어요 혼자 갔지만 매우매우 ...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 합산할 컬럼 리스트
sum_columns = [
    '예약 없이 이용', '예약 후 이용', '포장배달', '바로 입장', '10분 이내',
    '30분 이내', '30분 이상', '1시간 이상', '2시간 이상', '데이트',
    '친목', '나들이', '여행', '일상', '기념일', '회식', '가족모임',
    '비지니스', '연인・배우자', '친구', '지인・동료', '혼자', '아이',
    '부모님', '반려동물', '친척・형제', '기타'
]

# 'name'과 'address'를 기준으로 그룹화
grouped_df = df.groupby(['name', 'address'], as_index=False).first()  # 나머지 컬럼은 첫 번째 값을 유지

# 합산한 값 계산
sums = df.groupby(['name', 'address'])[sum_columns].sum().reset_index()

# 합산 결과를 원래 DataFrame에 결합
for column in sum_columns:
    grouped_df[column] = sums[column]

In [ ]:
grouped_df

,name,address,label,NAVER_name,category,review_num,reviews,review,예약 없이 이용,예약 후 이용,...,비지니스,연인・배우자,친구,지인・동료,혼자,아이,부모님,반려동물,친척・형제,기타
0,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,하얗게 불태웠다,0,0,...,0,0,0,0,0,0,0,0,0,0
1,일구칠사뼈감탕,제주 제주시 영평동 2169-3번지 지하1층 B103호,1,일구칠사뼈감탕,감자탕,35,예약 없이 이용 바로 입장 친목 친구 처음 방문했는데 음식 신선도며 맛이며 부족한게...,처음 방문했는데 음식 신선도며 맛이며 부족한게 하나 없이 정말 맛있게 잘 먹었습니다...,16,0,...,0,2,2,2,10,2,0,0,0,0
2,일도가공,제주 제주시 일도일동 1333-2번지 1층,1,속숨한제주,가공식품,29,특색있어요🥂 / 술종류 다양하고 사장님 왕친절하세요~^^\n택배도된다하니 다 먹고 ...,특색있어요,0,0,...,0,0,0,0,0,0,0,0,0,0
3,일도돈,제주 제주시 일도이동 378-4번지 동명빌딩 1층,1,조가네 숯불돈갈비,"육류,고기요리",15,예약 후 이용 바로 입장 데이트 연인・배우자 조가네숯불돈갈비에 가려다가 주차할 수 ...,조가네숯불돈갈비에 가려다가 주차할 수 있는 공간은 뒷편 골목길에 주차하고 왔어요 아...,0,2,...,0,2,0,0,0,0,0,0,0,0
4,일도바다장어구이,제주 제주시 일도이동 392-1번지,1,일도바다장어구이,"장어,먹장어요리",17,"바로 입장 일상 연인・배우자 통영바다장어 한가지 메뉴 1인분에 23,000원\n바닷...",통영바다장어 한가지 메뉴 인분에 원바닷장어라 가시가 있기는 한데 연하고 부들부들한 ...,8,0,...,0,4,2,2,2,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094,히든,제주 서귀포시 서귀동 321-5번지 1층,1,히든,"맥주,호프",19,예약 없이 이용 바로 입장 가족모임 부모님 숙소 근처에 있는 맥주집! 조용하고 테이...,숙소 근처에 있는 맥주집 조용하고 테이블 넓은 곳 찾다가 발견했는데 간단하게 맥주마...,1,0,...,0,0,1,0,0,1,1,0,0,0
2095,히로,제주 제주시 조천읍 함덕리 1001번지 103호,1,히로,일식당,9,예약 없이 이용 바로 입장 데이트 연인・배우자 가게 간판만 보고 들어간 곳!!!\n...,가게 간판만 보고 들어간 곳제주 여행에서 알려지지 않은 작은 가게에 맛집찾은 기분은...,9,0,...,0,4,4,0,0,1,0,0,0,0
2096,히치하이커스라운지,제주 제주시 이도이동 1771-13번지 지하1층,1,히치하이커스라운지,바(BAR),108,예약 없이 이용 바로 입장 친목 친구 오랜만에 모인 친구들과 한잔하러 방문했다가 여...,오랜만에 모인 들과 한잔하러 방문했다가 여러잔 걸치고 나왔습니다마성의 술집 시그니처...,59,1,...,0,18,36,4,15,0,1,0,0,0
2097,히토리,제주 서귀포시 서홍동 280-11번지 1층,1,히토리,이자카야,67,"예약 후 이용 바로 입장 여행 친구 세상에, 신선한 제주도의 해산물로 사장님이 바로...",세상에 신선한 제주도의 해산물로 사장님이 바로바로 만들어주세요오픈키친으로 보고 듣는...,7,26,...,0,22,10,3,1,0,1,0,0,0


In [ ]:
# name과 address를 기준으로 그룹화
def collect_reviews(group):
    # label이 0인 리뷰 리스트
    review_list_0 = group.loc[group['label'] == 0, 'review'].tolist()
    # label이 1인 리뷰 리스트
    review_list_1 = group.loc[group['label'] == 1, 'review'].tolist()

    # 그룹 내 하나의 행에만 review_list_0, review_list_1 추가
    group = group.iloc[0:1].copy()
    group['review_list_0'] = [review_list_0]
    group['review_list_1'] = [review_list_1]

    return group

# 그룹화하고 각 그룹에 대해 review_list 생성 및 중복 제거
df_result = df.groupby(['name', 'address'], group_keys=False).apply(collect_reviews)

# 필요 없는 review 컬럼 제거
df_result = df_result.drop(columns=['review'])

df_result

<ipython-input-19-702582269a71>:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_result = df.groupby(['name', 'address'], group_keys=False).apply(collect_reviews)


,label,name,address,NAVER_name,category,review_num,reviews,예약 없이 이용,예약 후 이용,포장배달,...,친구,지인・동료,혼자,아이,부모님,반려동물,친척・형제,기타,review_list_0,review_list_1
0,1,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,0,0,0,...,0,0,0,0,0,0,0,0,[],"[하얗게 불태웠다, 굿, 이색적인곳, 닭발 주먹밥 매콤하니 좋아요, 월드컵 관람하러..."
6,1,일구칠사뼈감탕,제주 제주시 영평동 2169-3번지 지하1층 B103호,일구칠사뼈감탕,감자탕,35,예약 없이 이용 바로 입장 친목 친구 처음 방문했는데 음식 신선도며 맛이며 부족한게...,1,0,0,...,1,0,0,0,0,0,0,0,"[함바식당 국, 집밥, 또또간집, 함바식당 국]",[처음 방문했는데 음식 신선도며 맛이며 부족한게 하나 없이 정말 맛있게 잘 먹었습니...
41,1,일도가공,제주 제주시 일도일동 1333-2번지 1층,속숨한제주,가공식품,29,특색있어요🥂 / 술종류 다양하고 사장님 왕친절하세요~^^\n택배도된다하니 다 먹고 ...,0,0,0,...,0,0,0,0,0,0,0,0,"[돼지감자칩, 돼지감자칩, 돼지감자칩, 돼지감자칩]","[특색있어요, 술종류 다양하고 사장님 왕친절하세요택배도된다하니 다 먹고 재주문할려구..."
70,1,일도돈,제주 제주시 일도이동 378-4번지 동명빌딩 1층,조가네 숯불돈갈비,"육류,고기요리",15,예약 후 이용 바로 입장 데이트 연인・배우자 조가네숯불돈갈비에 가려다가 주차할 수 ...,0,1,0,...,0,0,0,0,0,0,0,0,[],[조가네숯불돈갈비에 가려다가 주차할 수 있는 공간은 뒷편 골목길에 주차하고 왔어요 ...
85,1,일도바다장어구이,제주 제주시 일도이동 392-1번지,일도바다장어구이,"장어,먹장어요리",17,"바로 입장 일상 연인・배우자 통영바다장어 한가지 메뉴 1인분에 23,000원\n바닷...",0,0,0,...,0,0,0,0,0,0,0,0,"[불친절 가성비가 안좋았어요, 불친절 가성비가 안좋았어요]",[통영바다장어 한가지 메뉴 인분에 원바닷장어라 가시가 있기는 한데 연하고 부들부들한...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122974,1,히든,제주 서귀포시 서귀동 321-5번지 1층,히든,"맥주,호프",19,예약 없이 이용 바로 입장 가족모임 부모님 숙소 근처에 있는 맥주집! 조용하고 테이...,1,0,0,...,0,0,0,0,1,0,0,0,"[스아메리카노 사 먹으러 갔는데 머신기 고장으로 청포도 스무디 주문했어용, 카푸치노...",[숙소 근처에 있는 맥주집 조용하고 테이블 넓은 곳 찾다가 발견했는데 간단하게 맥주...
122993,1,히로,제주 제주시 조천읍 함덕리 1001번지 103호,히로,일식당,9,예약 없이 이용 바로 입장 데이트 연인・배우자 가게 간판만 보고 들어간 곳!!!\n...,1,0,0,...,0,0,0,0,0,0,0,0,[츄릅],[가게 간판만 보고 들어간 곳제주 여행에서 알려지지 않은 작은 가게에 맛집찾은 기분...
123002,1,히치하이커스라운지,제주 제주시 이도이동 1771-13번지 지하1층,히치하이커스라운지,바(BAR),108,예약 없이 이용 바로 입장 친목 친구 오랜만에 모인 친구들과 한잔하러 방문했다가 여...,1,0,0,...,1,0,0,0,0,0,0,0,"[칵테일이 참맛있네요, , ]",[오랜만에 모인 들과 한잔하러 방문했다가 여러잔 걸치고 나왔습니다마성의 술집 시그니...
123100,1,히토리,제주 서귀포시 서홍동 280-11번지 1층,히토리,이자카야,67,"예약 후 이용 바로 입장 여행 친구 세상에, 신선한 제주도의 해산물로 사장님이 바로...",0,1,0,...,1,0,0,0,0,0,0,0,[],[세상에 신선한 제주도의 해산물로 사장님이 바로바로 만들어주세요오픈키친으로 보고 듣...


In [ ]:
df_result = df_result[['name', 'address', 'NAVER_name', 'category', 'review_num',
       'reviews', 'review_list_0', 'review_list_1']]

In [ ]:
df_result

,name,address,NAVER_name,category,review_num,reviews,review_list_0,review_list_1
0,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,[],"[하얗게 불태웠다, 굿, 이색적인곳, 닭발 주먹밥 매콤하니 좋아요, 월드컵 관람하러..."
6,일구칠사뼈감탕,제주 제주시 영평동 2169-3번지 지하1층 B103호,일구칠사뼈감탕,감자탕,35,예약 없이 이용 바로 입장 친목 친구 처음 방문했는데 음식 신선도며 맛이며 부족한게...,"[함바식당 국, 집밥, 또또간집, 함바식당 국]",[처음 방문했는데 음식 신선도며 맛이며 부족한게 하나 없이 정말 맛있게 잘 먹었습니...
41,일도가공,제주 제주시 일도일동 1333-2번지 1층,속숨한제주,가공식품,29,특색있어요🥂 / 술종류 다양하고 사장님 왕친절하세요~^^\n택배도된다하니 다 먹고 ...,"[돼지감자칩, 돼지감자칩, 돼지감자칩, 돼지감자칩]","[특색있어요, 술종류 다양하고 사장님 왕친절하세요택배도된다하니 다 먹고 재주문할려구..."
70,일도돈,제주 제주시 일도이동 378-4번지 동명빌딩 1층,조가네 숯불돈갈비,"육류,고기요리",15,예약 후 이용 바로 입장 데이트 연인・배우자 조가네숯불돈갈비에 가려다가 주차할 수 ...,[],[조가네숯불돈갈비에 가려다가 주차할 수 있는 공간은 뒷편 골목길에 주차하고 왔어요 ...
85,일도바다장어구이,제주 제주시 일도이동 392-1번지,일도바다장어구이,"장어,먹장어요리",17,"바로 입장 일상 연인・배우자 통영바다장어 한가지 메뉴 1인분에 23,000원\n바닷...","[불친절 가성비가 안좋았어요, 불친절 가성비가 안좋았어요]",[통영바다장어 한가지 메뉴 인분에 원바닷장어라 가시가 있기는 한데 연하고 부들부들한...
...,...,...,...,...,...,...,...,...
122974,히든,제주 서귀포시 서귀동 321-5번지 1층,히든,"맥주,호프",19,예약 없이 이용 바로 입장 가족모임 부모님 숙소 근처에 있는 맥주집! 조용하고 테이...,"[스아메리카노 사 먹으러 갔는데 머신기 고장으로 청포도 스무디 주문했어용, 카푸치노...",[숙소 근처에 있는 맥주집 조용하고 테이블 넓은 곳 찾다가 발견했는데 간단하게 맥주...
122993,히로,제주 제주시 조천읍 함덕리 1001번지 103호,히로,일식당,9,예약 없이 이용 바로 입장 데이트 연인・배우자 가게 간판만 보고 들어간 곳!!!\n...,[츄릅],[가게 간판만 보고 들어간 곳제주 여행에서 알려지지 않은 작은 가게에 맛집찾은 기분...
123002,히치하이커스라운지,제주 제주시 이도이동 1771-13번지 지하1층,히치하이커스라운지,바(BAR),108,예약 없이 이용 바로 입장 친목 친구 오랜만에 모인 친구들과 한잔하러 방문했다가 여...,"[칵테일이 참맛있네요, , ]",[오랜만에 모인 들과 한잔하러 방문했다가 여러잔 걸치고 나왔습니다마성의 술집 시그니...
123100,히토리,제주 서귀포시 서홍동 280-11번지 1층,히토리,이자카야,67,"예약 후 이용 바로 입장 여행 친구 세상에, 신선한 제주도의 해산물로 사장님이 바로...",[],[세상에 신선한 제주도의 해산물로 사장님이 바로바로 만들어주세요오픈키친으로 보고 듣...


In [ ]:
# df_result와 grouped_df를 merge
merged_df = df_result.merge(grouped_df, on=['name', 'address'], how='left', suffixes=('', '_grouped'))

merged_df

,name,address,NAVER_name,category,review_num,reviews,review_list_0,review_list_1,label,NAVER_name_grouped,...,비지니스,연인・배우자,친구,지인・동료,혼자,아이,부모님,반려동물,친척・형제,기타
0,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,[],"[하얗게 불태웠다, 굿, 이색적인곳, 닭발 주먹밥 매콤하니 좋아요, 월드컵 관람하러...",1,1943 서귀포명동로점,...,0,0,0,0,0,0,0,0,0,0
1,일구칠사뼈감탕,제주 제주시 영평동 2169-3번지 지하1층 B103호,일구칠사뼈감탕,감자탕,35,예약 없이 이용 바로 입장 친목 친구 처음 방문했는데 음식 신선도며 맛이며 부족한게...,"[함바식당 국, 집밥, 또또간집, 함바식당 국]",[처음 방문했는데 음식 신선도며 맛이며 부족한게 하나 없이 정말 맛있게 잘 먹었습니...,1,일구칠사뼈감탕,...,0,2,2,2,10,2,0,0,0,0
2,일도가공,제주 제주시 일도일동 1333-2번지 1층,속숨한제주,가공식품,29,특색있어요🥂 / 술종류 다양하고 사장님 왕친절하세요~^^\n택배도된다하니 다 먹고 ...,"[돼지감자칩, 돼지감자칩, 돼지감자칩, 돼지감자칩]","[특색있어요, 술종류 다양하고 사장님 왕친절하세요택배도된다하니 다 먹고 재주문할려구...",1,속숨한제주,...,0,0,0,0,0,0,0,0,0,0
3,일도돈,제주 제주시 일도이동 378-4번지 동명빌딩 1층,조가네 숯불돈갈비,"육류,고기요리",15,예약 후 이용 바로 입장 데이트 연인・배우자 조가네숯불돈갈비에 가려다가 주차할 수 ...,[],[조가네숯불돈갈비에 가려다가 주차할 수 있는 공간은 뒷편 골목길에 주차하고 왔어요 ...,1,조가네 숯불돈갈비,...,0,2,0,0,0,0,0,0,0,0
4,일도바다장어구이,제주 제주시 일도이동 392-1번지,일도바다장어구이,"장어,먹장어요리",17,"바로 입장 일상 연인・배우자 통영바다장어 한가지 메뉴 1인분에 23,000원\n바닷...","[불친절 가성비가 안좋았어요, 불친절 가성비가 안좋았어요]",[통영바다장어 한가지 메뉴 인분에 원바닷장어라 가시가 있기는 한데 연하고 부들부들한...,1,일도바다장어구이,...,0,4,2,2,2,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094,히든,제주 서귀포시 서귀동 321-5번지 1층,히든,"맥주,호프",19,예약 없이 이용 바로 입장 가족모임 부모님 숙소 근처에 있는 맥주집! 조용하고 테이...,"[스아메리카노 사 먹으러 갔는데 머신기 고장으로 청포도 스무디 주문했어용, 카푸치노...",[숙소 근처에 있는 맥주집 조용하고 테이블 넓은 곳 찾다가 발견했는데 간단하게 맥주...,1,히든,...,0,0,1,0,0,1,1,0,0,0
2095,히로,제주 제주시 조천읍 함덕리 1001번지 103호,히로,일식당,9,예약 없이 이용 바로 입장 데이트 연인・배우자 가게 간판만 보고 들어간 곳!!!\n...,[츄릅],[가게 간판만 보고 들어간 곳제주 여행에서 알려지지 않은 작은 가게에 맛집찾은 기분...,1,히로,...,0,4,4,0,0,1,0,0,0,0
2096,히치하이커스라운지,제주 제주시 이도이동 1771-13번지 지하1층,히치하이커스라운지,바(BAR),108,예약 없이 이용 바로 입장 친목 친구 오랜만에 모인 친구들과 한잔하러 방문했다가 여...,"[칵테일이 참맛있네요, , ]",[오랜만에 모인 들과 한잔하러 방문했다가 여러잔 걸치고 나왔습니다마성의 술집 시그니...,1,히치하이커스라운지,...,0,18,36,4,15,0,1,0,0,0
2097,히토리,제주 서귀포시 서홍동 280-11번지 1층,히토리,이자카야,67,"예약 후 이용 바로 입장 여행 친구 세상에, 신선한 제주도의 해산물로 사장님이 바로...",[],[세상에 신선한 제주도의 해산물로 사장님이 바로바로 만들어주세요오픈키친으로 보고 듣...,1,히토리,...,0,22,10,3,1,0,1,0,0,0


In [ ]:
merged_df.columns

Index(['name', 'address', 'NAVER_name', 'category', 'review_num', 'reviews',
       'review_list_0', 'review_list_1', 'label', 'NAVER_name_grouped',
       'category_grouped', 'review_num_grouped', 'reviews_grouped', 'review',
       '예약 없이 이용', '예약 후 이용', '포장배달', '바로 입장', '10분 이내', '30분 이내', '30분 이상',
       '1시간 이상', '2시간 이상', '데이트', '친목', '나들이', '여행', '일상', '기념일', '회식', '가족모임',
       '비지니스', '연인・배우자', '친구', '지인・동료', '혼자', '아이', '부모님', '반려동물', '친척・형제',
       '기타'],
      dtype='object')

In [ ]:
merged_df = merged_df[['name', 'address', 'NAVER_name', 'category', 'review_num', 'reviews',
       'review_list_0', 'review_list_1','예약 없이 이용', '예약 후 이용', '포장배달', '바로 입장', '10분 이내', '30분 이내', '30분 이상',
       '1시간 이상', '2시간 이상', '데이트', '친목', '나들이', '여행', '일상', '기념일', '회식', '가족모임',
       '비지니스', '연인・배우자', '친구', '지인・동료', '혼자', '아이', '부모님', '반려동물', '친척・형제',
       '기타']]

In [ ]:
merged_df[:50]

,name,address,NAVER_name,category,review_num,reviews,review_list_0,review_list_1,예약 없이 이용,예약 후 이용,...,비지니스,연인・배우자,친구,지인・동료,혼자,아이,부모님,반려동물,친척・형제,기타
0,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,하얗게 불태웠다 / 굿 / 이색적인곳 / 닭발 주먹밥 매콤하니 좋아요 / 월드컵 관...,[],"[하얗게 불태웠다, 굿, 이색적인곳, 닭발 주먹밥 매콤하니 좋아요, 월드컵 관람하러...",0,0,...,0,0,0,0,0,0,0,0,0,0
1,일구칠사뼈감탕,제주 제주시 영평동 2169-3번지 지하1층 B103호,일구칠사뼈감탕,감자탕,35,예약 없이 이용 바로 입장 친목 친구 처음 방문했는데 음식 신선도며 맛이며 부족한게...,"[함바식당 국, 집밥, 또또간집, 함바식당 국]",[처음 방문했는데 음식 신선도며 맛이며 부족한게 하나 없이 정말 맛있게 잘 먹었습니...,16,0,...,0,2,2,2,10,2,0,0,0,0
2,일도가공,제주 제주시 일도일동 1333-2번지 1층,속숨한제주,가공식품,29,특색있어요🥂 / 술종류 다양하고 사장님 왕친절하세요~^^\n택배도된다하니 다 먹고 ...,"[돼지감자칩, 돼지감자칩, 돼지감자칩, 돼지감자칩]","[특색있어요, 술종류 다양하고 사장님 왕친절하세요택배도된다하니 다 먹고 재주문할려구...",0,0,...,0,0,0,0,0,0,0,0,0,0
3,일도돈,제주 제주시 일도이동 378-4번지 동명빌딩 1층,조가네 숯불돈갈비,"육류,고기요리",15,예약 후 이용 바로 입장 데이트 연인・배우자 조가네숯불돈갈비에 가려다가 주차할 수 ...,[],[조가네숯불돈갈비에 가려다가 주차할 수 있는 공간은 뒷편 골목길에 주차하고 왔어요 ...,0,2,...,0,2,0,0,0,0,0,0,0,0
4,일도바다장어구이,제주 제주시 일도이동 392-1번지,일도바다장어구이,"장어,먹장어요리",17,"바로 입장 일상 연인・배우자 통영바다장어 한가지 메뉴 1인분에 23,000원\n바닷...","[불친절 가성비가 안좋았어요, 불친절 가성비가 안좋았어요]",[통영바다장어 한가지 메뉴 인분에 원바닷장어라 가시가 있기는 한데 연하고 부들부들한...,8,0,...,0,4,2,2,2,0,0,0,2,0
5,일도전복,제주 제주시 일도이동 109-17번지 1층,어마장장 해물뚝배기,전복요리,743,예약 없이 이용 바로 입장 여행 연인・배우자 해물뚝배기가 먹고 싶어서 방문했는데 기...,"[음돈가스가제일맛나다메인은 걍 그럼, 맛은 그닥전복 네개 넣어주네요, 리뷰좋길래 갔...",[해물뚝배기가 먹고 싶어서 방문했는데 기대 이상이였어요반찬도 하나하나 다 맛있고 정...,68,0,...,0,30,8,4,2,16,16,0,10,0
6,일도전복 제주공항점,제주 제주시 도두일동 1742번지 1층,일도전복 제주공항점,전복요리,377,예약 없이 이용 바로 입장 일상 연인・배우자 일도전복은 제주도민이 찾는 전복 맛집이...,[],[일도전복은 제주도민이 찾는 전복 맛집이에요 제주공항점도 생겼다해서 방문해보았어요 ...,138,54,...,0,85,28,10,12,44,41,0,16,0
7,일도집,제주 제주시 일도이동 365-14번지 1층,생마차 제주일도점,이자카야,22,예약 없이 이용 바로 입장 친목 지인・동료 모든 메뉴가 전체적으로 다 좋았습니당🤤\...,"[저녁먹고 차로 방문, 닭날개가 아니고 봉이에요엄청 쪼꼬맘 ㅎ사람 시끌벅적 차로 드...",[모든 메뉴가 전체적으로 다 좋았습니당다만 엄청 시끄러워서 이야기 나누기는 조금 힘...,40,0,...,0,12,12,4,0,0,0,0,0,2
8,일도차돌,제주 제주시 일도이동 62-6번지 1층,순백순두부,두부요리,55,예약 없이 이용 바로 입장 일상 친척・형제 매장이 청결해요 / 예약 없이 이용 바로...,"[, 살면서 가본 음식점 중 최악 입니다리뷰 속 친절한 사장님은 없으셨어요매우 무례...","[매장이 청결해요, 매장이 깨끗합니다, 제주 가성비 호텔에서 자고 바로옆 식당에서 ...",14,0,...,0,0,0,2,2,0,0,0,8,0
9,일도해장국,제주 제주시 일도이동 85번지 1층,일도해장국,해장국,195,예약 없이 이용 바로 입장 데이트 연인・배우자 아침에 해장하러 방문했는데 국물이 깔...,"[예전에 비해 맛이 많이 아쉬웠어요 , 해장국에서 비닐나옴 그냥 죄송합니다 한마디띡...",[아침에 해장하러 방문했는데 국물이 깔끔하면서 해장국마다 선지가 많이 들어가있어서 ...,72,30,...,0,44,10,18,20,16,14,0,6,0


In [ ]:
merged_df['reviews'] = merged_df['review_list_0']+	merged_df['review_list_1']

<ipython-input-26-003c240e4444>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['reviews'] = merged_df['review_list_0']+	merged_df['review_list_1']


In [ ]:
merged_df

,name,address,NAVER_name,category,review_num,reviews,review_list_0,review_list_1,예약 없이 이용,예약 후 이용,...,비지니스,연인・배우자,친구,지인・동료,혼자,아이,부모님,반려동물,친척・형제,기타
0,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,"[하얗게 불태웠다, 굿, 이색적인곳, 닭발 주먹밥 매콤하니 좋아요, 월드컵 관람하러...",[],"[하얗게 불태웠다, 굿, 이색적인곳, 닭발 주먹밥 매콤하니 좋아요, 월드컵 관람하러...",0,0,...,0,0,0,0,0,0,0,0,0,0
1,일구칠사뼈감탕,제주 제주시 영평동 2169-3번지 지하1층 B103호,일구칠사뼈감탕,감자탕,35,"[함바식당 국, 집밥, 또또간집, 함바식당 국, 처음 방문했는데 음식 신선도며 맛이...","[함바식당 국, 집밥, 또또간집, 함바식당 국]",[처음 방문했는데 음식 신선도며 맛이며 부족한게 하나 없이 정말 맛있게 잘 먹었습니...,16,0,...,0,2,2,2,10,2,0,0,0,0
2,일도가공,제주 제주시 일도일동 1333-2번지 1층,속숨한제주,가공식품,29,"[돼지감자칩, 돼지감자칩, 돼지감자칩, 돼지감자칩, 특색있어요, 술종류 다양하고 사...","[돼지감자칩, 돼지감자칩, 돼지감자칩, 돼지감자칩]","[특색있어요, 술종류 다양하고 사장님 왕친절하세요택배도된다하니 다 먹고 재주문할려구...",0,0,...,0,0,0,0,0,0,0,0,0,0
3,일도돈,제주 제주시 일도이동 378-4번지 동명빌딩 1층,조가네 숯불돈갈비,"육류,고기요리",15,[조가네숯불돈갈비에 가려다가 주차할 수 있는 공간은 뒷편 골목길에 주차하고 왔어요 ...,[],[조가네숯불돈갈비에 가려다가 주차할 수 있는 공간은 뒷편 골목길에 주차하고 왔어요 ...,0,2,...,0,2,0,0,0,0,0,0,0,0
4,일도바다장어구이,제주 제주시 일도이동 392-1번지,일도바다장어구이,"장어,먹장어요리",17,"[불친절 가성비가 안좋았어요, 불친절 가성비가 안좋았어요, 통영바다장어 한가지 메뉴...","[불친절 가성비가 안좋았어요, 불친절 가성비가 안좋았어요]",[통영바다장어 한가지 메뉴 인분에 원바닷장어라 가시가 있기는 한데 연하고 부들부들한...,8,0,...,0,4,2,2,2,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094,히든,제주 서귀포시 서귀동 321-5번지 1층,히든,"맥주,호프",19,"[스아메리카노 사 먹으러 갔는데 머신기 고장으로 청포도 스무디 주문했어용, 카푸치노...","[스아메리카노 사 먹으러 갔는데 머신기 고장으로 청포도 스무디 주문했어용, 카푸치노...",[숙소 근처에 있는 맥주집 조용하고 테이블 넓은 곳 찾다가 발견했는데 간단하게 맥주...,1,0,...,0,0,1,0,0,1,1,0,0,0
2095,히로,제주 제주시 조천읍 함덕리 1001번지 103호,히로,일식당,9,"[츄릅, 가게 간판만 보고 들어간 곳제주 여행에서 알려지지 않은 작은 가게에 맛집찾...",[츄릅],[가게 간판만 보고 들어간 곳제주 여행에서 알려지지 않은 작은 가게에 맛집찾은 기분...,9,0,...,0,4,4,0,0,1,0,0,0,0
2096,히치하이커스라운지,제주 제주시 이도이동 1771-13번지 지하1층,히치하이커스라운지,바(BAR),108,"[칵테일이 참맛있네요, , , 오랜만에 모인 들과 한잔하러 방문했다가 여러잔 걸치고...","[칵테일이 참맛있네요, , ]",[오랜만에 모인 들과 한잔하러 방문했다가 여러잔 걸치고 나왔습니다마성의 술집 시그니...,59,1,...,0,18,36,4,15,0,1,0,0,0
2097,히토리,제주 서귀포시 서홍동 280-11번지 1층,히토리,이자카야,67,[세상에 신선한 제주도의 해산물로 사장님이 바로바로 만들어주세요오픈키친으로 보고 듣...,[],[세상에 신선한 제주도의 해산물로 사장님이 바로바로 만들어주세요오픈키친으로 보고 듣...,7,26,...,0,22,10,3,1,0,1,0,0,0


In [ ]:
merged_df.to_csv('./cleaned_review_v2.csv', encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/Crawling/지우/data/cleaned_review_v2.csv')
raw = df.copy()
df.head(3)

,name,address,NAVER_name,category,review_num,reviews,review_list_0,review_list_1,예약 없이 이용,예약 후 이용,...,비지니스,연인・배우자,친구,지인・동료,혼자,아이,부모님,반려동물,친척・형제,기타
0,일구사삼(1943)서귀포명동로점,제주 서귀포시 서귀동 408-2번지 2층,1943 서귀포명동로점,요리주점,7,"['하얗게 불태웠다', '굿', '이색적인곳', '닭발 주먹밥 매콤하니 좋아요', ...",[],"['하얗게 불태웠다', '굿', '이색적인곳', '닭발 주먹밥 매콤하니 좋아요', ...",0,0,...,0,0,0,0,0,0,0,0,0,0
1,일구칠사뼈감탕,제주 제주시 영평동 2169-3번지 지하1층 B103호,일구칠사뼈감탕,감자탕,35,"['함바식당 국', '집밥', '또또간집', '함바식당 국', '처음 방문했는데 음...","['함바식당 국', '집밥', '또또간집', '함바식당 국']",['처음 방문했는데 음식 신선도며 맛이며 부족한게 하나 없이 정말 맛있게 잘 먹었습...,16,0,...,0,2,2,2,10,2,0,0,0,0
2,일도가공,제주 제주시 일도일동 1333-2번지 1층,속숨한제주,가공식품,29,"['돼지감자칩', '돼지감자칩', '돼지감자칩', '돼지감자칩', '특색있어요', ...","['돼지감자칩', '돼지감자칩', '돼지감자칩', '돼지감자칩']","['특색있어요', '술종류 다양하고 사장님 왕친절하세요택배도된다하니 다 먹고 재주문...",0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 24.2 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
from collections import Counter
import re


In [ ]:
df1 = df[:1000]
df2 = df[1000:]

In [ ]:
import json
# 말뭉치 생성
corpus_1 = "".join(df1['reviews'].tolist())
corpus_1
# nouns 리스트를 JSON 파일로 저장
with open('./corpus_list_1.json', 'w', encoding='utf-8') as f:
    json.dump(corpus_1, f, ensure_ascii=False)



import json
# 말뭉치 생성
corpus_2 = "".join(df2['reviews'].tolist())
corpus_2
# nouns 리스트를 JSON 파일로 저장
with open('./corpus_list_2.json', 'w', encoding='utf-8') as f:
    json.dump(corpus_2, f, ensure_ascii=False)

In [ ]:
import json

# JSON 파일에서 nouns 리스트 불러오기
with open('./corpus_list_1.json', 'r', encoding='utf-8') as f:
    corpus_1 = json.load(f)

corpus_1

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import json

# JSON 파일에서 nouns 리스트 불러오기
with open('./corpus_list_2.json', 'r', encoding='utf-8') as f:
    corpus_2 = json.load(f)

corpus_2

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 전체 말뭉치(corpus)에서 명사 형태소 추출
okt = Okt()

# 명사 추출
nouns_1 = okt.nouns(corpus_1)

nouns_1

['불',
 '굿',
 '이색',
 '곳',
 '닭발',
 '주먹밥',
 '매콤',
 '월드컵',
 '관람',
 '음식',
 '기본',
 '이상',
 '샴페인',
 '불',
 '굿',
 '이색',
 '곳',
 '닭발',
 '주먹밥',
 '매콤',
 '월드컵',
 '관람',
 '음식',
 '기본',
 '이상',
 '샴페인',
 '함바',
 '식당',
 '국',
 '집밥',
 '또또',
 '집',
 '함바',
 '식당',
 '국',
 '처음',
 '방문',
 '음식',
 '신선도',
 '맛',
 '하나',
 '정말',
 '직원',
 '덕분',
 '더',
 '기분',
 '식사',
 '수',
 '재',
 '방문',
 '약속',
 '맘데',
 '함바',
 '식당',
 '반찬',
 '가지',
 '수가',
 '맛',
 '기류',
 '돼지',
 '불고기',
 '돈까스',
 '찜닭',
 '가격',
 '대비',
 '요추',
 '점심',
 '뷔페',
 '집',
 '싼값',
 '점심',
 '먹기',
 '가성',
 '비',
 '맛집',
 '밥',
 '반찬',
 '고기',
 '모두',
 '질',
 '재',
 '방문',
 '퍼',
 '넘흐',
 '맛',
 '요',
 '한식',
 '뷔페',
 '최고',
 '매장',
 '쾌적',
 '제주',
 '가격',
 '이정',
 '반찬',
 '이면',
 '완전',
 '집',
 '또또',
 '집',
 '또또',
 '집',
 '밥',
 '굿',
 '굿',
 '가성',
 '비',
 '사장',
 '가격',
 '도착',
 '음식',
 '굿',
 '가성',
 '비',
 '한식',
 '집',
 '굿',
 '굿',
 '굿',
 '굿',
 '점심',
 '부폐집',
 '가성',
 '비',
 '처음',
 '방문',
 '음식',
 '신선도',
 '맛',
 '하나',
 '정말',
 '직원',
 '덕분',
 '더',
 '기분',
 '식사',
 '수',
 '재',
 '방문',
 '약속',
 '맘데',
 '함바',
 '식당',
 '반찬',
 '가지',
 '수가',
 '맛',
 '

In [ ]:
# 전체 말뭉치(corpus)에서 명사 형태소 추출
okt = Okt()

# 명사 추출
nouns_2 = okt.nouns(corpus_2)

nouns_2

['그냥',
 '저',
 '만원',
 '끼',
 '레시',
 '맛',
 '후기',
 '계란후라이',
 '하나',
 '반찬',
 '가성',
 '비도',
 '정식',
 '고기',
 '별로',
 '동네',
 '현지민',
 '맛집',
 '맛집',
 '음식',
 '서귀포',
 '이제야',
 '생각',
 '매장',
 '반찬',
 '하나',
 '하나',
 '음식',
 '솜씨',
 '듯',
 '요정',
 '때',
 '한번',
 '다이빙',
 '항상',
 '점심',
 '곳',
 '양도',
 '반찬',
 '저녁',
 '한식',
 '고기',
 '쌈',
 '생선',
 '바삭',
 '만원',
 '행복',
 '성비',
 '가성',
 '비',
 '도민',
 '맛집',
 '점심시간',
 '가기',
 '서귀포',
 '중',
 '혼밥',
 '식당',
 '식당',
 '도중',
 '혼밥',
 '도민',
 '맛집',
 '얘기',
 '방문',
 '가격',
 '혼자',
 '눈',
 '치안',
 '무엇',
 '맛',
 '저',
 '수육',
 '밥상',
 '갖가지',
 '찬',
 '수육',
 '쌈',
 '채소',
 '생선',
 '구이',
 '오늘',
 '국',
 '배',
 '꽉',
 '경험',
 '원래',
 '반찬',
 '하나',
 '밥',
 '스타일',
 '식사',
 '혼밥',
 '가성',
 '비',
 '분',
 '점',
 '추천',
 '수육',
 '맛',
 '올레길',
 '배',
 '불리',
 '반찬',
 '국',
 '밥',
 '무한리필',
 '만원',
 '행복',
 '계란후라이',
 '설명',
 '이만',
 '정식',
 '찾기',
 '요정',
 '땐',
 '충남',
 '식당',
 '음식',
 '년월',
 '정식',
 '천원',
 '요새',
 '물가',
 '반찬',
 '생선',
 '반찬',
 '정식',
 '집기',
 '찬',
 '국',
 '밥',
 '리필',
 '서귀포시',
 '점심',
 '곳',
 '제주',
 '여행',
 '약',
 '주',
 '중',
 '제일',
 '밥집',
 '시간',
 '것',

In [ ]:
nouns = nouns_1 + nouns_2
nouns

['불',
 '굿',
 '이색',
 '곳',
 '닭발',
 '주먹밥',
 '매콤',
 '월드컵',
 '관람',
 '음식',
 '기본',
 '이상',
 '샴페인',
 '불',
 '굿',
 '이색',
 '곳',
 '닭발',
 '주먹밥',
 '매콤',
 '월드컵',
 '관람',
 '음식',
 '기본',
 '이상',
 '샴페인',
 '함바',
 '식당',
 '국',
 '집밥',
 '또또',
 '집',
 '함바',
 '식당',
 '국',
 '처음',
 '방문',
 '음식',
 '신선도',
 '맛',
 '하나',
 '정말',
 '직원',
 '덕분',
 '더',
 '기분',
 '식사',
 '수',
 '재',
 '방문',
 '약속',
 '맘데',
 '함바',
 '식당',
 '반찬',
 '가지',
 '수가',
 '맛',
 '기류',
 '돼지',
 '불고기',
 '돈까스',
 '찜닭',
 '가격',
 '대비',
 '요추',
 '점심',
 '뷔페',
 '집',
 '싼값',
 '점심',
 '먹기',
 '가성',
 '비',
 '맛집',
 '밥',
 '반찬',
 '고기',
 '모두',
 '질',
 '재',
 '방문',
 '퍼',
 '넘흐',
 '맛',
 '요',
 '한식',
 '뷔페',
 '최고',
 '매장',
 '쾌적',
 '제주',
 '가격',
 '이정',
 '반찬',
 '이면',
 '완전',
 '집',
 '또또',
 '집',
 '또또',
 '집',
 '밥',
 '굿',
 '굿',
 '가성',
 '비',
 '사장',
 '가격',
 '도착',
 '음식',
 '굿',
 '가성',
 '비',
 '한식',
 '집',
 '굿',
 '굿',
 '굿',
 '굿',
 '점심',
 '부폐집',
 '가성',
 '비',
 '처음',
 '방문',
 '음식',
 '신선도',
 '맛',
 '하나',
 '정말',
 '직원',
 '덕분',
 '더',
 '기분',
 '식사',
 '수',
 '재',
 '방문',
 '약속',
 '맘데',
 '함바',
 '식당',
 '반찬',
 '가지',
 '수가',
 '맛',
 '

In [ ]:
import json

# nouns 리스트를 JSON 파일로 저장
with open('./nouns_list.json', 'w', encoding='utf-8') as f:
    json.dump(nouns, f, ensure_ascii=False)

In [ ]:
import json

# JSON 파일에서 nouns 리스트 불러오기
with open('./nouns_list.json', 'r', encoding='utf-8') as f:
    nouns = json.load(f)

# 출력 결과 확인
print(nouns)


Output hidden; open in https://colab.research.google.com to view.